In [108]:
'''
import zipfile
with zipfile.ZipFile("hw1__data.zip","r") as zip_ref:
    zip_ref.extractall("nwhw1_data")
'''    

'\nimport zipfile\nwith zipfile.ZipFile("hw1__data.zip","r") as zip_ref:\n    zip_ref.extractall("nwhw1_data")\n'

In [109]:
import os
raw_data_base = "/home/p88101029/nwhw2_data/dataset1/"
os.makedirs(raw_data_base, exist_ok=True)

In [110]:
listoffolder=os.listdir(raw_data_base)

In [111]:
import csv
import numpy as np
import torch

# 開啟 CSV 檔案
with open(raw_data_base+'content.csv') as csvfile:
    attri=[]
  # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)

  # 以迴圈輸出每一列
    for row in rows:
        attri.append(list(map(int,row[0].split())))

attri = np.array(attri)
attri = attri[np.argsort(attri[:,0])][:,1:]
attri = torch.tensor(attri).to(dtype=torch.float)

In [112]:
with open(raw_data_base+'train.csv') as csvfile:
    train=[]
  # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
    next(rows, None)
  # 以迴圈輸出每一列
    for row in rows:
        train.append(row)
        
train = np.array(train)
train = train[:,1:].astype(int)
train = train[np.argsort(train[:,0])]
trainpos = torch.tensor(train[train[:,2]==1][:,0:2].T)
trainneg = torch.tensor(train[train[:,2]==0][:,0:2].T)
trainans = torch.tensor(train[:,2].T)
train = torch.tensor(train[:,0:2].T)

with open(raw_data_base+'test.csv') as csvfile:
    test=[]
  # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
    next(rows, None)
  # 以迴圈輸出每一列
    for row in rows:
        test.append(row)
        
test = np.array(test)
edgename = test[:,0]
test = test[:,1:].astype(int)
test = torch.tensor(test.T)

In [113]:
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn
from torch_geometric.data import Data

In [114]:
fullset = Data(x=attri, edge_index=trainpos)

In [115]:
fullset

Data(x=[2708, 1433], edge_index=[2, 4324])

In [116]:
train.shape

torch.Size([2, 8686])

In [117]:
trainposid = np.random.choice(trainpos.shape[1], trainpos.shape[1])
trainnegid = np.random.choice(trainneg.shape[1], trainneg.shape[1])

In [118]:
trainposedge = trainpos[:,trainposid[:int(trainpos.shape[1]*0.9)]]
testposedge = trainpos[:,trainposid[int(trainpos.shape[1]*0.9):]]
trainnegedge = trainneg[:,trainnegid[:int(trainneg.shape[1]*0.9)]]
testnegedge = trainneg[:,trainnegid[int(trainneg.shape[1]*0.9):]]

In [119]:
train_pos_g = Data(x=attri, edge_index=trainposedge)
train_neg_g = Data(x=attri, edge_index=trainnegedge)
test_pos_g = Data(x=attri, edge_index=testposedge)
test_neg_g = Data(x=attri, edge_index=testnegedge)

In [120]:
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv
from torch.nn import Linear
from torch_geometric.utils import degree

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv(in_channels, out_channels)
        self.conv2a = GATConv(in_channels, hidden_channels)
        self.conv2b = GATConv(hidden_channels, out_channels)
        self.lin = Linear(out_channels*2,int(out_channels/2),bias=True)
        self.lin2 = Linear(int(out_channels/2),1,bias=True)
        
    def encode(self, x, edge_index):
        fl = self.conv1(x, edge_index)
        sla = self.conv2a(x, edge_index).relu()
        sl = self.conv2b(sla, edge_index)
        return torch.cat([fl, sl], dim=1)

    def decode(self, z, edge_index):
        #matrix = (z[edge_index[0]] * z[edge_index[1]]) 
        matrix = (z[edge_index[0]] - z[edge_index[1]]).pow(2)
        pred = self.lin(matrix).relu()
        pred = self.lin2(pred)      
        return pred


In [121]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from torch_geometric.utils import to_undirected
import copy

def train_link_predictor(
    model, train_pos_g, train_neg_g, test_pos_g, test_neg_g, optimizer, criterion, n_epochs=1000
):
    train_pos_g.edge_index = to_undirected(train_pos_g.edge_index)
    train_neg_g.edge_index = to_undirected(train_neg_g.edge_index)
    test_pos_g.edge_index = to_undirected(test_pos_g.edge_index)
    test_neg_g.edge_index = to_undirected(test_neg_g.edge_index)
    trainposedges = train_pos_g.edge_index.shape[1]
    trainnegedges = train_neg_g.edge_index.shape[1]
    testposedges = test_pos_g.edge_index.shape[1]
    testnegedges = test_neg_g.edge_index.shape[1]
    trainedgesindex = torch.cat([train_pos_g.edge_index, train_neg_g.edge_index],dim=-1)
    trainedge_label = torch.cat([torch.ones(trainposedges),torch.zeros(trainnegedges)], dim=0)
    testedgesindex = torch.cat([test_pos_g.edge_index, test_neg_g.edge_index],dim=-1)
    testedge_label = torch.cat([torch.ones(testposedges),torch.zeros(testnegedges)], dim=0)
    
    best_score = 0
    counter = 0
    stop_loss = 0
    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_pos_g.x, train_pos_g.edge_index)
        out = model.decode(z, trainedgesindex).view(-1)
        loss = criterion(out, trainedge_label)
        loss.backward()
        optimizer.step()

        if epoch % 5 == 0:
            out = eval_link_predictor(model, train_pos_g, testedgesindex)
            val_auc = roc_auc_score(testedge_label.cpu().numpy(), out.cpu().numpy())
            val_pre = precision_score(testedge_label.cpu().numpy(), torch.round(out).cpu().numpy())
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}, Val PRE: {val_pre:.3f}")
            score = (val_auc*2 + val_pre)/3
            if score > best_score:
                counter = 0
                best_score = score
                stop_loss = loss
                save_model = copy.deepcopy(model)
            else:
                counter +=1
            if counter == 20:
                break
    return save_model, stop_loss


@torch.no_grad()
def eval_link_predictor(model, train_pos_g, testedgesindex):
    model.eval()
    z = model.encode(train_pos_g.x, train_pos_g.edge_index)
    out = model.decode(z, testedgesindex).view(-1).sigmoid()
    return out

In [122]:
model = Net(attri.shape[1], int(128), int(64))
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()

model, stop_loss = train_link_predictor(model, train_pos_g, train_neg_g, test_pos_g, test_neg_g, optimizer, criterion)

/home/p88101029/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 005, Train Loss: 0.697, Val AUC: 0.508, Val PRE: 0.000


/home/p88101029/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 010, Train Loss: 0.696, Val AUC: 0.637, Val PRE: 0.000


/home/p88101029/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch: 220, Train Loss: 0.442, Val AUC: 0.959, Val PRE: 0.000


/home/p88101029/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 225, Train Loss: 0.439, Val AUC: 0.959, Val PRE: 0.000


/home/p88101029/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 230, Train Loss: 0.435, Val AUC: 0.960, Val PRE: 0.000
Epoch: 235, Train Loss: 0.432, Val AUC: 0.960, Val PRE: 1.000
Epoch: 240, Train Loss: 0.429, Val AUC: 0.960, Val PRE: 1.000
Epoch: 245, Train Loss: 0.427, Val AUC: 0.960, Val PRE: 1.000
Epoch: 250, Train Loss: 0.424, Val AUC: 0.960, Val PRE: 1.000
Epoch: 255, Train Loss: 0.421, Val AUC: 0.960, Val PRE: 1.000
Epoch: 260, Train Loss: 0.417, Val AUC: 0.959, Val PRE: 1.000
Epoch: 265, Train Loss: 0.414, Val AUC: 0.959, Val PRE: 0.966
Epoch: 270, Train Loss: 0.411, Val AUC: 0.958, Val PRE: 0.972
Epoch: 275, Train Loss: 0.407, Val AUC: 0.958, Val PRE: 0.950
Epoch: 280, Train Loss: 0.403, Val AUC: 0.958, Val PRE: 0.953
Epoch: 285, Train Loss: 0.399, Val AUC: 0.958, Val PRE: 0.956
Epoch: 290, Train Loss: 0.395, Val AUC: 0.957, Val PRE: 0.958
Epoch: 295, Train Loss: 0.391, Val AUC: 0.956, Val PRE: 0.940
Epoch: 300, Train Loss: 0.387, Val AUC: 0.956, Val PRE: 0.943
Epoch: 305, Train Loss: 0.383, Val AUC: 0.955, Val PRE: 0.948
Epoch: 3

In [123]:
def train_link_predictor( model, fullset, train, trainans, optimizer, criterion, stop_loss=0, n_epochs=1000):   
    fullset.edge_index = to_undirected(fullset.edge_index)
    train = torch.cat((train,torch.stack((train[1,:],train[0,:]))),dim=1)
    trainans = torch.cat((trainans,trainans),dim=0)
    best_loss = 1
    counter = 0
    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(fullset.x, fullset.edge_index)
        out = model.decode(z, train).view(-1)
        loss = criterion(out, trainans)
        loss.backward()
        optimizer.step()

        if epoch % 5 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}")
            if loss < best_loss and loss > 0.001:
                counter = 0
                best_loss = loss
            else:
                counter +=1
            if counter == 20 or loss < stop_loss:
                break
    return model


@torch.no_grad()
def eval_link_predictor(model, fullset, test):
    fullset.edge_index = to_undirected(fullset.edge_index)
    model.eval()
    z = model.encode(fullset.x, fullset.edge_index)
    out = model.decode(z, test).view(-1).sigmoid()
    return out

In [124]:
model = Net(attri.shape[1], int(128), int(64))
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()

model = train_link_predictor(model, fullset, train, trainans.to(dtype=torch.float), optimizer, criterion, stop_loss)

Epoch: 005, Train Loss: 0.693
Epoch: 010, Train Loss: 0.693
Epoch: 015, Train Loss: 0.692
Epoch: 020, Train Loss: 0.692
Epoch: 025, Train Loss: 0.691
Epoch: 030, Train Loss: 0.690
Epoch: 035, Train Loss: 0.689
Epoch: 040, Train Loss: 0.688
Epoch: 045, Train Loss: 0.687
Epoch: 050, Train Loss: 0.686
Epoch: 055, Train Loss: 0.684
Epoch: 060, Train Loss: 0.682
Epoch: 065, Train Loss: 0.679
Epoch: 070, Train Loss: 0.676
Epoch: 075, Train Loss: 0.672
Epoch: 080, Train Loss: 0.668
Epoch: 085, Train Loss: 0.662
Epoch: 090, Train Loss: 0.656
Epoch: 095, Train Loss: 0.648
Epoch: 100, Train Loss: 0.640
Epoch: 105, Train Loss: 0.630
Epoch: 110, Train Loss: 0.618
Epoch: 115, Train Loss: 0.606
Epoch: 120, Train Loss: 0.592
Epoch: 125, Train Loss: 0.578
Epoch: 130, Train Loss: 0.564
Epoch: 135, Train Loss: 0.551
Epoch: 140, Train Loss: 0.537
Epoch: 145, Train Loss: 0.526
Epoch: 150, Train Loss: 0.515
Epoch: 155, Train Loss: 0.505
Epoch: 160, Train Loss: 0.497
Epoch: 165, Train Loss: 0.489
Epoch: 170

In [125]:
testpred = eval_link_predictor(model, fullset, test)

In [126]:
import pandas as pd

ids = edgename
prob = testpred
upload = {'id': ids, 'prob': prob} 

path = 'upload1.csv'
df = pd.DataFrame(upload)
df.to_csv(path, index=False)